In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Text

## Bag of words

A widely used technique in NLP (natural language processing). It's a great approach to start with for any text-based problem. It's also the basis of many other more advanced methods. 

### Tokenization and transformation

The splitting of text into pieces is known as tokenization. The most common way to split is on words, but in some cases (for example in character based langauges) you may want to split on character or split on pairs or groups of words or even something more advanced. 

